In [65]:
import pandas as pd
import numpy as np
import altair as alt

In [66]:
df = pd.read_csv("performance_results.csv")
df

,query_name,method,iteration,time,n_rows,id_size
0,Attributes,TempoQL,1,5.224552,1000,1000
1,Attributes,SQL,1,2.478078,1000,1000
2,Events,TempoQL,1,5.649733,88998,1000
3,Events,SQL,1,4.933126,88998,1000
4,String Operations,TempoQL,1,3.364550,18808,1000
...,...,...,...,...,...,...
283,Rolling Difference,SQL,3,33.992630,1087429,50000
284,Imputing Missing Values,TempoQL,3,16.240138,1113535,50000
285,Imputing Missing Values,SQL,3,45.238080,1113539,50000
286,Carrying Values Forward,TempoQL,3,18.473943,206181,50000


In [ ]:
grouped_df = (df[df['method'] == 'TempoQL'].set_index(["query_name", "iteration", "id_size"])['time'] - df[df['method'] == 'SQL'].set_index(["query_name", "iteration", "id_size"])['time']).reset_index()
chart = alt.Chart(grouped_df)
(chart.mark_errorbar().encode(
    y=alt.Y("time:Q", scale=alt.Scale(type='symlog', domain=[-100, 100])),
    x=alt.X("id_size:O"),
    color=alt.Color("query_name:N", sort=df['query_name'].unique()),
) + chart.mark_line().encode(
    y=alt.Y("mean(time):Q", scale=alt.Scale(type='symlog', domain=[-100, 100])),
    x=alt.X("id_size:O"),
    color=alt.Color("query_name:N", sort=df['query_name'].unique()),
)).properties(width=400)

alt.LayerChart(...)

In [118]:
grouped_df = (df[df['method'] == 'TempoQL'].set_index(["query_name", "iteration", "id_size"])['time'] - df[df['method'] == 'SQL'].set_index(["query_name", "iteration", "id_size"])['time']).rename('time_diff').reset_index()
grouped_df['sql_time'] = df[df['method'] == 'SQL']['time'].reset_index(drop=True)
grouped_df['tempo_time'] = df[df['method'] == 'TempoQL']['time'].reset_index(drop=True)

chart = alt.Chart(grouped_df)
((chart.mark_rect().encode(
    x=alt.X("id_size:O", axis=alt.Axis(title="Number of Patients", labelAngle=0)),
    y=alt.Y("query_name:N", sort=grouped_df['query_name'].unique(), axis=alt.Axis(title=None)),
    color=alt.Color("mean_time:Q", 
                    title="TempoQL Time – SQL Time (s)", 
                    bin=alt.Bin(extent=[-90, 90], step=20, maxbins=9, nice=False),
                    scale=alt.Scale(scheme='redblue', 
                                    reverse=True)),
) + chart.mark_text(
    align='center',
    baseline='middle',
    fontSize=12
).encode(
    x=alt.X("id_size:O"),
    y=alt.Y("query_name:N", sort=grouped_df['query_name'].unique()),
    text=alt.Text("time_desc:N"),
    color=alt.condition("abs(datum.mean_time) > 40", alt.value("white"), alt.value("black"))
)).transform_aggregate(
    mean_time="mean(time_diff)",
    mean_sql_time="mean(sql_time)",
    mean_tempo_time="mean(tempo_time)",
    groupby=["id_size", "query_name"]
).transform_calculate(
    time_desc="format(datum.mean_tempo_time, '.1f') + ' v. ' + format(datum.mean_sql_time, '.1f')"
).properties(width=400, title="TempoQL v. SQL Execution Time"))

alt.LayerChart(...)

In [107]:
grouped_df

,query_name,iteration,id_size,time_diff,time_desc
0,Attributes,1,1000,2.746474,2.5 v. 5.2
1,Events,1,1000,0.716607,4.9 v. 5.6
2,String Operations,1,1000,0.745880,2.6 v. 3.4
3,Discretizing Observations,1,1000,0.927316,3.7 v. 4.6
4,Patient-Level Aggregation,1,1000,11.831398,2.4 v. 14.3
...,...,...,...,...,...
139,Aggregating Existence at Event Times,3,50000,3.827533,2.9 v. 6.7
140,Aggregating Counts at Event Times,3,50000,-50.633407,88.3 v. 37.7
141,Rolling Difference,3,50000,-20.540953,34.0 v. 13.5
142,Imputing Missing Values,3,50000,-28.997942,45.2 v. 16.2


In [86]:
(df[df['method'] == 'TempoQL'].set_index(["query_name", "iteration", "id_size"])['time'] - df[df['method'] == 'SQL'].set_index(["query_name", "iteration", "id_size"])['time']).rename('time_diff').reset_index()

,query_name,iteration,id_size,time_diff
0,Attributes,1,1000,2.746474
1,Events,1,1000,0.716607
2,String Operations,1,1000,0.745880
3,Discretizing Observations,1,1000,0.927316
4,Patient-Level Aggregation,1,1000,11.831398
...,...,...,...,...
139,Aggregating Existence at Event Times,3,50000,3.827533
140,Aggregating Counts at Event Times,3,50000,-50.633407
141,Rolling Difference,3,50000,-20.540953
142,Imputing Missing Values,3,50000,-28.997942


In [ ]:
grouped_df = ((df[df['method'] == 'TempoQL'].set_index(["query_name", "iteration", "id_size"])['time'] - df[df['method'] == 'SQL'].set_index(["query_name", "iteration", "id_size"])['time']) / df[df['method'] == 'SQL'].set_index(["query_name", "iteration", "id_size"])['time'] * 100).reset_index()

chart = alt.Chart(grouped_df)
(chart.mark_rect().encode(
    x=alt.X("id_size:O", axis=alt.Axis(title="Number of Patients", labelAngle=0)),
    y=alt.Y("query_name:N", sort=df['query_name'].unique(), axis=alt.Axis(title=None)),
    color=alt.Color("mean_time:Q", 
                    title="TempoQL Time (% Increase from SQL Time)", 
                    bin=alt.Bin(extent=[-1000, 1000], step=200, maxbins=10, nice=False),
                    scale=alt.Scale(scheme='redblue', 
                                    reverse=True)),
) + chart.mark_text(
    align='center',
    baseline='middle',
    fontSize=12
).encode(
    x=alt.X("id_size:O"),
    y=alt.Y("query_name:N", sort=df['query_name'].unique()),
    text=alt.Text("mean_time:Q", format="+.0f"),
    color=alt.condition("abs(datum.mean_time) > 800", alt.value("white"), alt.value("black"))
)).transform_aggregate(
    mean_time="mean(time)",
    groupby=["id_size", "query_name"]
).properties(width=200)